## GRAPH

### Linked Directed Graph

In [ ]:
class LinkedDirectedGraph(AbstractCollection):
    def __init__(self, sourceCollection = None):
        self.edgeCount = 0
        self.vertices = dict() # Dictionary of vertices
        AbstractCollection.__init__(self, sourceCollection)
    
    def addVertex(self, label):
        """Adds a vertex with the given label to the graph."""
        self.vertices[label] = LinkedVertex(label)
        self.size += 1

    def removeVertex(self, label):
        """Returns True if the vertex was removed, or False
        otherwise."""
        removedVertex = self.vertices.pop(label, None)
        if removedVertex is None:
            return False
        # Examine all other vertices to remove edges
        # directed at the removed vertex
        for vertex in self.getVertices():
            if vertex.removeEdgeTo(removedVertex):
                self.edgeCount -= 1
        # Examine all edges from the removed vertex to others
        for edge in removedVertex.incidentEdges():
            self.edgeCount -= 1
        self.size -= 1
        return True

    def addEdge(self, fromLabel, toLabel, weight):
        """Connects the vertices with an edge with the given weight."""
        fromVertex = self.getVertex(fromLabel)
        toVertex = self.getVertex(toLabel)
        fromVertex.addEdgeTo(toVertex, weight)
        self.edgeCount += 1

    def getEdge(self, fromLabel, toLabel):
        """Returns the edge connecting the two vertices, or None if
        no edge exists."""
        fromVertex = self.getVertex(fromLabel)
        toVertex = self.getVertex(toLabel)
        return fromVertex.getEdgeTo(toVertex)

    def removeEdge(self, fromLabel, toLabel):
        """Returns True if the edge was removed, or False otherwise."""
        fromVertex = self.getVertex(fromLabel)
        toVertex = self.getVertex(toLabel)
        edgeRemovedFlg = fromVertex.removeEdgeTo(toVertex)
        if edgeRemovedFlg:
            self.edgeCount -= 1
        return edgeRemovedFlg

    def edges(self):
        """Supports iteration over the edges in the graph."""
        result = set()
        for vertex in self.getVertices():
            edges = vertex.incidentEdges()
            result = result.union(set(edges))
        return iter(result)

### Linked Vertex

In [ ]:
class LinkedVertex(object):
    def __init__(self, label):
        self.label = label
        self.edgeList = list()
        self.mark = False

    def setLabel(self, label, g):
        """Sets the vertex’s label to label."""
        g.vertices.pop(self.label, None)
        g.vertices[label] = self
        self.label = label

    def neighboringVertices(self):
        """Returns the neighboring vertices of this vertex."""
        vertices = list()
        for edge in self.edgeList:
            vertices.append(edge.getOtherVertex(self))
        return iter(vertices)

    def removeEdgeTo(self, toVertex):
        """Returns True if the edge exists and is removed,
        or False otherwise."""
        edge = LinkedEdge(self, toVertex)
        if edge in self.edgeList:
            self.edgeList.remove(edge)
            return True
        else:
            return False

    def __hash__(self):
        """Supports hashing on a vertex."""
        return hash(self.label)

### Linked Edge

In [ ]:
class LinkedEdge(object):
    def __init__(self, fromVertex, toVertex, weight = None):
        self.vertex1 = fromVertex
        self.vertex2 = toVertex
        self.weight = weight
        self.mark = False

    def __eq__(self, other):
        """Two edges are equal if they connect
        the same vertices."""
        if self is other: return True
        if type(self) != type(other): return False
        return self.vertex1 == other.vertex1 and \
            self.vertex2 == other.vertex2 and \
            self.weight == other.weight